In [1]:
from selenium.webdriver.common.by import By
from SimulatorCommunicator import setup, switchToIframe, scrollToTheBottom, updateURL, clickElement

import networkx as nx
import random
import matplotlib.pyplot as plt
import re

In [2]:
def getAllClickables(driver):
    elements = []

    elements_a = driver.find_elements(By.TAG_NAME, 'a')
    for element in elements_a:
        if element.is_displayed() and element.is_enabled() and element.rect.__len__() != 0:  # excluding all the hidden links
            # print(element.get_property('attributes')[0].items(), "\n")

            data = {}
            data['x'] = element.rect['x']
            data['y'] = element.rect['y']
            data['width'] = element.rect['width']
            data['height'] = element.rect['height']
            data['outer_html'] = element.get_attribute('outerHTML')
            data['href'] = element.get_dom_attribute('href')
            data['href_full'] = element.get_attribute('href')
            data['class'] = element.get_attribute('class')
            data['name'] = element.text
            elements.append(data)
            # break
    return elements

In [3]:
# checks one agains a list
def isSimilar(query, reference):
    points = 0
    # relative url match
    if(query[0]) == reference[0]:
        points += 1

    # absolute url match
    if(query[1]) == reference[1]:
        points += 1
  
    # class match  
    if(query[2]) == reference[2]:
        points += 1 

    return points >= 2

# check one url
def checkIfIn(query, urlList):
    for url in urlList:
        if isSimilar(query, url):
            return True
    
    return False

In [4]:
l1 = [
     ('#', 'index.html#', 'nav-link\n                js-top-drawer-click')
]

l2 = [
     ('collections/womens-new-arrivals.html', 'collections/womens-new-arrivals.html', 'topnav-mobile-link'),
     ('collections/womens.html', 'collections/womens.html', 'topnav-mobile-link'), 
     ('collections/kids.html', 'collections/kids.html', 'topnav-mobile-link'),
     ('collections/mens.html', 'collections/mens.html', 'topnav-mobile-link'),
]

l3 = [
     ('collections/womens.html', 'collections/womens.html', 'topnav-mobile-link'),
     ('collections/womens-new-arrivals.html', 'collections/womens-new-arrivals.html', 'topnav-mobile-link'),
     ('collections/womens-socks.html', 'collections/womens-socks.html', 'topnav-mobile-link'),
     ('collections/womens-tights-and-leggings.html', 'collections/womens-tights-and-leggings.html', 'topnav-mobile-link'),
     ('collections/womens-clothing.html', 'collections/womens-clothing.html', 'topnav-mobile-link'),
     ('collections/womens-bags.html', 'collections/womens-bags.html', 'topnav-mobile-link'),
     ('collections/womens-sale.html', 'collections/womens-sale.html', 'topnav-mobile-link'),

     ('collections/kids.html', 'collections/kids.html', 'topnav-mobile-link'),
     ('collections/kids-new-arrivals.html', 'collections/kids-new-arrivals.html', 'topnav-mobile-link'),
     ('collections/kids-socks.html', 'collections/kids-socks.html', 'topnav-mobile-link'),
     ('collections/kids-tights-and-leggings.html', 'collections/kids-tights-and-leggings.html', 'topnav-mobile-link'),
     ('collections/kids-sale.html', 'collections/kids-sale.html', 'topnav-mobile-link'),

     ('collections/mens.html', 'collections/mens.html', 'topnav-mobile-link'),
     ('collections/mens-new-arrivals-1.html', 'collections/mens-new-arrivals-1.html', 'topnav-mobile-link'),
     ('collections/mens-socks.html', 'collections/mens-socks.html', 'topnav-mobile-link'),
     ('collections/mens-sale.html', 'collections/mens-sale.html', 'topnav-mobile-link'),
]

eliminate = [
     ('collections/mens.html', 'collections/mens.html', 'topnav-mobile-link'),
     ('collections/womens-tights-and-leggings.html', 'collections/womens-tights-and-leggings.html', 'subnav-link\n                        '),
     ('collections/womens-bags.html', 'collections/womens-bags.html', 'subnav-link\n                        '),
     ('collections/womens-gift-card.html', 'collections/womens-gift-card.html', 'subnav-link\n                        '),
     ('collections/womens-sale.html', 'collections/womens-sale.html', 'subnav-link\n                        '),
     ('collections/womens-sporty-series.html', 'collections/womens-sporty-series.html', 'subnav-link\n                        '),
     ('collections/womens-conversational-crews.html', 'collections/womens-conversational-crews.html', 'subnav-link\n                        '),
     ('collections/womens-new-arrivals.html', 'collections/womens-new-arrivals.html', ''),
     ('collections/kids-gift-card.html', 'collections/kids-gift-card.html', 'subnav-link\n                        '),
     ('collections/kids-sale.html', 'collections/kids-sale.html', 'subnav-link\n                        '),
     ('collections/kids-new-arrivals.html', 'collections/kids-new-arrivals.html', ''),
     ('collections/mens-new-arrivals-1.html', 'collections/mens-new-arrivals-1.html', '')
]

In [8]:
base = "http://localhost:4200/assets/crawled/hansel/hanselfrombasel.com/"

def collectData(n_data):
    driver = setup()
    switchToIframe(driver)

    for i in range(n_data):
        scrollToTheBottom(driver)
        # clickables = getAllClickables(driver)

        clickables = [clickable for clickable in getAllClickables(driver) if not checkIfIn((clickable['href'], clickable['href_full'].replace(base,""), clickable['class']), eliminate)]
        # print(clickables)
        
        chosenLink = random.choice(clickables[:])
        print(chosenLink)

        clickElement(driver, chosenLink['x'], chosenLink['y'])
        # break


collectData(5)

{'x': 425.859375, 'y': 5020.234375, 'width': 103, 'height': 20, 'outer_html': '<a href="pages/press.html" class="footer-link">\n            Press\n          </a>', 'href': 'pages/press.html', 'href_full': 'http://localhost:4200/assets/crawled/hansel/hanselfrombasel.com/pages/press.html', 'class': 'footer-link', 'name': 'Press'}
{'x': 705.84375, 'y': 64.953125, 'width': 35, 'height': 21, 'outer_html': '<a href="../search.html" class="nav-link\n                nav-link-mobile">\n        <!--?xml version="1.0" encoding="utf-8"?-->\n<!-- Generator: Adobe Illustrator 18.1.1, SVG Export Plug-In . SVG Version: 6.00 Build 0)  -->\n<svg version="1.1" class="icon icon-search" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" x="0px" y="0px" viewBox="0 0 20 20" enable-background="new 0 0 20 20" xml:space="preserve">\n<g>\n\t<path d="M6.6,7.5c0.3,0.1,0.5,0.4,0.7,0.6c0.3,0.2,0.5,0.5,0.8,0.7c0.2-0.2,0.4-0.4,0.6-0.6C8.5,7.9,8.1,7.6,7.8,7.3\n\t\tC7.7,7.2,7.4,6.9,7.3,6.9C7.1

In [10]:
a = set([(1,1,1), (2,2,2), ('collections/mens.html', 'collections/mens.html', 'topnav-mobile-link')])
b = set([('collections/mens.html', 'collections/mens.html', 'topnav-mobile-link')])

a.difference(b)

{(1, 1, 1), (2, 2, 2)}

In [ ]:
# driver = setup(False)
# switchToIframe(driver)
# [(click['href'], click['href_full'].replace(base,""), click['class']) for click in getAllClickables(driver)]

# data = {
#     'href':'assets/crawled/hansel/hanselfrombasel.com/index.html',
#     'href_full':'assets/crawled/hansel/hanselfrombasel.com/index.html',
#     'x': 0,
#     'y': 0}


In [11]:
# Unused

['pages/shipping.html__FREE SHIPPING ON US ORDERS $75+',
 '#_nav-link\n                js-top-drawer-click_Group Copymenu Created With Sketch.',
 'account/login.html#sign-in_nav-link_',
 'index.html_nav-lockup_',
 'search.html_nav-link\n                nav-link-mobile_',
 'cart.html_nav-link\n                nav-link-mobile\n                js-drawer-toggle_',
 'collections/womens-new-arrivals.html_topnav-mobile-link_New Arrivals',
 "collections/womens.html_topnav-mobile-link_Women's",
 "collections/kids.html_topnav-mobile-link_Kid's",
 'collections/kids.html_topnav-mobile-link_All',
 'collections/kids-new-arrivals.html_topnav-mobile-link_New Arrivals',
 'collections/kids-socks.html_topnav-mobile-link_Socks',
 'collections/kids-tights-and-leggings.html_topnav-mobile-link_Tights and Leggings',
 'collections/kids-sale.html_topnav-mobile-link_SALE',
 "collections/mens.html_topnav-mobile-link_Men's",
 'collections/mens.html_topnav-mobile-link_All',
 'collections/mens-new-arrivals-1.html_to

# Unused

In [73]:
def collectData(n_data):
    driver = setup()
    switchToIframe(driver)

    for i in range(n_data):
        # scrollToTheBottom(driver)
        # clickables = getAllClickables(driver)
        # chosenLink = random.choice(clickables[10:])
        # print(chosenLink)

        clickElement(driver, 414, 3000)
        break

        

# collectData(5)

In [ ]:
G = nx.DiGraph()
G.add_node('a', time='a')
G.add_node("f", time="f")
G.add_edge('f', 'f')
# showGraph(G)




In [7]:
# G.nodes(data=True)

In [ ]:
def showGraph(G):
    options = {
        "font_size": 36,
        "node_size": 3000,
        "node_color": "white",
        "edgecolors": "black",
        "linewidths": 5,
        "width": 5,
    }

    nx.draw_networkx(G)
    # Set margins for the axes so that nodes aren't clipped
    ax = plt.gca()
    ax.margins(0.20)
    plt.axis("off")
    plt.show()




# visitedLinks = set(['http://localhost:4200/assets/crawled/hansel/hanselfrombasel.com/cart.html'])

# def collectData(driver, prevRoot, root, meunuClicked=False):
#     # visitedLinks.add(root['href_full'].replace("#", ""))

#     clickElement(driver, x=root['x'], y=root['y'])
#     scrollToTheBottom(driver)
#     clickables = getAllClickables(driver)

#     # links with # have to be handled
#     clickables_diff = set([clickable['href_full'].replace("#", "") for clickable in clickables]).difference(visitedLinks)

#     clickables = [clickable for clickable in clickables if clickable['href_full'].replace("#", "") in clickables_diff]

#     if clickables.__len__() == 0:
#         return

#     for clickable in clickables:
#         print(clickable['href'], clickable['href_full'])
#         collectData(driver, root, clickable, False)

#         print("LEVEL UP")
#         # maybe parent is missing-- correct it
#         updateURL(driver, prevRoot['href_full'])
        